# Resource Validation

> Validate resource availability before job execution and determine appropriate actions

In [ ]:
#| default_exp core.validation

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional
from enum import Enum
from dataclasses import dataclass

from cjm_fasthtml_resources.core.manager import (
    ResourceStatus,
    ResourceConflict,
    WorkerState
)

## Validation Actions

The validation system returns different actions based on resource availability:

In [ ]:
#| export
class ValidationAction(Enum):
    """Actions that can be taken based on validation results."""
    PROCEED = "proceed"  # Resource available, proceed with job
    RELOAD_PLUGIN = "reload_plugin"  # Reload plugin (for switching plugins or resource configs)
    WAIT_FOR_JOB = "wait_for_job"  # Wait for current job to complete (same worker type)
    WAIT_FOR_OTHER_WORKER = "wait_for_other_worker"  # Wait for different worker type to finish
    USER_INTERVENTION = "user_intervention"  # External process conflict, need user action
    ABORT = "abort"  # Cannot proceed

## Validation Result

The result object contains all information needed to make decisions about job execution:

In [ ]:
#| export
@dataclass
class ValidationResult:
    """Result of resource validation."""
    action: ValidationAction
    can_proceed: bool
    message: str
    conflict: Optional[ResourceConflict] = None
    current_worker: Optional[WorkerState] = None
    plugin_name_to_reload: Optional[str] = None  # Plugin name to reload
    new_config: Optional[Dict[str, Any]] = None  # Config for reload

In [ ]:
# Example: Creating a validation result
result = ValidationResult(
    action=ValidationAction.PROCEED,
    can_proceed=True,
    message="GPU available. Proceeding with job."
)

print(f"Action: {result.action.value}")
print(f"Can proceed: {result.can_proceed}")
print(f"Message: {result.message}")

Action: proceed
Can proceed: True
Message: GPU available. Proceeding with job.


## Resource Validation Function

This is the main validation function that coordinates resource checks and returns appropriate actions.

The validation logic handles several scenarios:

1. **CPU-based models**: Check system memory availability
2. **API-based models**: Skip resource validation
3. **GPU-based models**: Check GPU availability and handle conflicts
4. **Plugin switching**: Detect when plugins need to be reloaded
5. **Resource conflicts**: Identify conflicts with app workers or external processes

In [ ]:
#| export
def validate_resources_for_job(
    resource_manager,  # ResourceManager instance
    plugin_registry,  # Plugin registry protocol (has get_plugin, load_plugin_config methods)
    get_plugin_resource_requirements,  # Function: (plugin_id, config) -> Dict with requirements
    compare_plugin_resources,  # Function: (config1, config2) -> bool (same resource?)
    get_plugin_resource_identifier,  # Function: (config) -> str (resource ID)
    plugin_id: str,
    plugin_config: Optional[Dict[str, Any]] = None,
    worker_pid: Optional[int] = None,
    worker_type: str = "transcription",
    verbose: bool = False
) -> ValidationResult:
    """
    Validate if resources are available to run a job with the specified plugin.
    
    This function is dependency-injected with helper functions to avoid tight coupling
    with specific plugin registry implementations.
    
    Args:
        resource_manager: ResourceManager instance
        plugin_registry: Plugin registry with get_plugin, load_plugin_config methods
        get_plugin_resource_requirements: Function to get plugin requirements
        compare_plugin_resources: Function to compare plugin resources
        get_plugin_resource_identifier: Function to get resource ID
        plugin_id: Unique plugin ID
        plugin_config: Plugin configuration (will load if not provided)
        worker_pid: PID of the worker that will run the job (if known)
        worker_type: Type of worker (e.g., "transcription", "llm", "ollama")
        verbose: Whether to print verbose logging
    
    Returns:
        ValidationResult with action to take
    """
    if verbose:
        print(f"\n=== Resource Validation Started ===")
        print(f"Plugin ID: {plugin_id}")
        print(f"Worker PID: {worker_pid}")
        print(f"Worker Type: {worker_type}")

    # Get plugin resource requirements
    requirements = get_plugin_resource_requirements(plugin_id, plugin_config)
    if verbose:
        print(f"Plugin requirements: {requirements}")

    # Check system memory if using CPU
    if requirements['is_local'] and requirements['device'].lower() == 'cpu':
        if verbose:
            print("→ CPU-based model, checking system memory...")

        # TODO: Add comprehensive system memory validation
        # For now, just log current memory status
        try:
            import psutil
            mem = psutil.virtual_memory()
            if verbose:
                print(f"   System memory: {mem.percent}% used ({mem.used / (1024**3):.1f}GB / {mem.total / (1024**3):.1f}GB)")

            # Placeholder warning (can be enhanced with thresholds)
            if mem.percent > 85:
                if verbose:
                    print(f"   ⚠ System memory high: {mem.percent}% used")
        except ImportError:
            if verbose:
                print("   ⚠ psutil not available, skipping memory check")

        # For now, always proceed for CPU models
        if verbose:
            print("✓ System memory check passed (basic)")
            print("=== Resource Validation Complete ===\n")
        return ValidationResult(
            action=ValidationAction.PROCEED,
            can_proceed=True,
            message="CPU-based model. System memory check passed."
        )

    # If not a local model or doesn't use GPU, no validation needed
    if not requirements['is_local'] or not requirements['uses_gpu']:
        if verbose:
            print("✓ No GPU resources required")
            print("=== Resource Validation Complete ===\n")
        return ValidationResult(
            action=ValidationAction.PROCEED,
            can_proceed=True,
            message="No GPU resources required. Proceeding with job."
        )

    # Load plugin config if not provided
    if plugin_config is None:
        plugin_config = plugin_registry.load_plugin_config(plugin_id)

    # Get plugin metadata
    plugin_meta = plugin_registry.get_plugin(plugin_id)
    if not plugin_meta:
        return ValidationResult(
            action=ValidationAction.ABORT,
            can_proceed=False,
            message=f"Plugin {plugin_id} not found."
        )

    plugin_name = plugin_meta.name
    if verbose:
        print(f"Plugin name: {plugin_name}")

    # Check GPU availability
    gpu_conflict = resource_manager.check_gpu_availability()
    if verbose:
        print(f"GPU status: {gpu_conflict.status.value}")
        print(f"App PIDs using GPU: {gpu_conflict.app_pids}")
        print(f"External PIDs using GPU: {gpu_conflict.external_pids}")

    # Case 1: GPU is available (no processes using it)
    if gpu_conflict.status == ResourceStatus.AVAILABLE:
        if verbose:
            print("GPU Status: AVAILABLE")
        # Check if worker already has a plugin loaded
        if worker_pid:
            worker = resource_manager.get_worker_by_pid(worker_pid)
            if worker and worker.plugin_name:
                if verbose:
                    print(f"Worker has plugin loaded: {worker.plugin_name}")
                if worker.plugin_name == plugin_name:
                    # Same plugin - check if same plugin resource
                    if worker.config and compare_plugin_resources(worker.config, plugin_config):
                        # Same plugin, same resource - proceed
                        if verbose:
                            print(f"✓ Same plugin and resource already loaded")
                            print("Action: PROCEED")
                            print("=== Resource Validation Complete ===\n")
                        return ValidationResult(
                            action=ValidationAction.PROCEED,
                            can_proceed=True,
                            message="Plugin and resource already loaded. Proceeding with job.",
                            current_worker=worker
                        )
                    else:
                        # Same plugin, different resource - reload with new config
                        old_resource = get_plugin_resource_identifier(worker.config) if worker.config else "unknown"
                        new_resource = get_plugin_resource_identifier(plugin_config)
                        if verbose:
                            print(f"→ Resource change detected: {old_resource} → {new_resource}")
                            print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                            print("=== Resource Validation Complete ===\n")
                        return ValidationResult(
                            action=ValidationAction.RELOAD_PLUGIN,
                            can_proceed=True,
                            message=f"Reloading {plugin_name} with new resource configuration.",
                            current_worker=worker,
                            plugin_name_to_reload=plugin_name,
                            new_config=plugin_config
                        )
                else:
                    # Different plugin - reload to switch plugins
                    if verbose:
                        print(f"→ Plugin switch: {worker.plugin_name} → {plugin_name}")
                        print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                        print("=== Resource Validation Complete ===\n")
                    return ValidationResult(
                        action=ValidationAction.RELOAD_PLUGIN,
                        can_proceed=True,
                        message=f"Switching from {worker.plugin_name} to {plugin_name}.",
                        current_worker=worker,
                        plugin_name_to_reload=plugin_name,
                        new_config=plugin_config
                    )

        # GPU available, no plugin loaded - proceed
        if verbose:
            print("✓ No plugin loaded, GPU available")
            print("Action: PROCEED")
            print("=== Resource Validation Complete ===\n")
        return ValidationResult(
            action=ValidationAction.PROCEED,
            can_proceed=True,
            message="GPU available. Proceeding with job."
        )

    # Case 2: GPU busy with application process
    elif gpu_conflict.status == ResourceStatus.APP_BUSY:
        if verbose:
            print("GPU Status: APP_BUSY")
        # Find which worker is using GPU
        app_pids = gpu_conflict.app_pids
        if not app_pids:
            # Shouldn't happen, but handle gracefully
            if verbose:
                print("⚠ GPU reported as busy but no app PIDs found")
                print("Action: PROCEED (fallback)")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.PROCEED,
                can_proceed=True,
                message="GPU reported as busy but no app PIDs found. Proceeding."
            )

        # Get worker using GPU
        gpu_worker_pid = app_pids[0]  # Assuming single GPU, single worker
        if verbose:
            print(f"GPU in use by app worker PID: {gpu_worker_pid}")
        gpu_worker = resource_manager.get_worker_by_pid(gpu_worker_pid)

        if not gpu_worker:
            # Unknown worker, proceed cautiously
            if verbose:
                print(f"⚠ Unknown worker PID {gpu_worker_pid}")
                print("Action: PROCEED (cautiously)")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.PROCEED,
                can_proceed=True,
                message=f"GPU in use by app PID {gpu_worker_pid}. Proceeding anyway.",
                conflict=gpu_conflict
            )

        if verbose:
            print(f"Worker status: {gpu_worker.status}")
            print(f"Worker plugin: {gpu_worker.plugin_name}")

        # Check if it's the same worker we're going to use
        if worker_pid and gpu_worker_pid == worker_pid:
            if verbose:
                print("→ Same worker will be reused")
            # Same worker - check plugin and resource
            if gpu_worker.plugin_name == plugin_name:
                # Same plugin - check resource
                if gpu_worker.config and compare_plugin_resources(gpu_worker.config, plugin_config):
                    # Same plugin, same resource
                    if verbose:
                        print("✓ Same plugin and resource already loaded")
                        print("Action: PROCEED")
                        print("=== Resource Validation Complete ===\n")
                    return ValidationResult(
                        action=ValidationAction.PROCEED,
                        can_proceed=True,
                        message="Plugin and resource already loaded. Proceeding with job.",
                        current_worker=gpu_worker
                    )
                else:
                    # Same plugin, different resource - reload with new config
                    old_resource = get_plugin_resource_identifier(gpu_worker.config) if gpu_worker.config else "unknown"
                    new_resource = get_plugin_resource_identifier(plugin_config)
                    if verbose:
                        print(f"→ Resource change detected: {old_resource} → {new_resource}")
                        print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                        print("=== Resource Validation Complete ===\n")
                    return ValidationResult(
                        action=ValidationAction.RELOAD_PLUGIN,
                        can_proceed=True,
                        message=f"Reloading {plugin_name} with new resource configuration.",
                        current_worker=gpu_worker,
                        plugin_name_to_reload=plugin_name,
                        new_config=plugin_config
                    )
            else:
                # Different plugin - reload to switch
                if verbose:
                    print(f"→ Plugin switch: {gpu_worker.plugin_name} → {plugin_name}")
                    print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                    print("=== Resource Validation Complete ===\n")
                return ValidationResult(
                    action=ValidationAction.RELOAD_PLUGIN,
                    can_proceed=True,
                    message=f"Switching from {gpu_worker.plugin_name} to {plugin_name}.",
                    current_worker=gpu_worker,
                    plugin_name_to_reload=plugin_name,
                    new_config=plugin_config
                )

        # Different worker using GPU
        if verbose:
            print(f"→ Different worker is using GPU (type: {gpu_worker.worker_type})")

        # Check if it's a different worker TYPE
        if gpu_worker.worker_type != worker_type:
            if verbose:
                print(f"→ Cross-worker-type conflict: {gpu_worker.worker_type} ↔ {worker_type}")

            if gpu_worker.status == "running":
                # Different worker type is actively running
                if verbose:
                    print(f"✗ {gpu_worker.worker_type.capitalize()} worker is actively running")
                    print("Action: WAIT_FOR_OTHER_WORKER")
                    print("=== Resource Validation Complete ===\n")
                return ValidationResult(
                    action=ValidationAction.WAIT_FOR_OTHER_WORKER,
                    can_proceed=False,
                    message=f"GPU in use by running {gpu_worker.worker_type} job (PID {gpu_worker_pid}). Wait for completion or cancel that job.",
                    current_worker=gpu_worker,
                    conflict=gpu_conflict
                )
            else:
                # Different worker type is idle - could stop it
                if verbose:
                    print(f"⚠ {gpu_worker.worker_type.capitalize()} worker is idle but using GPU")
                    print("Action: WAIT_FOR_OTHER_WORKER")
                    print("=== Resource Validation Complete ===\n")
                return ValidationResult(
                    action=ValidationAction.WAIT_FOR_OTHER_WORKER,
                    can_proceed=False,
                    message=f"GPU in use by idle {gpu_worker.worker_type} worker (PID {gpu_worker_pid}). Stop that worker to free GPU.",
                    current_worker=gpu_worker,
                    conflict=gpu_conflict
                )

        # Same worker type but different instance
        if gpu_worker.status == "running":
            # Worker is actively running a job
            if verbose:
                print("✗ Worker is actively running a job")
                print("Action: WAIT_FOR_JOB")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.WAIT_FOR_JOB,
                can_proceed=False,
                message=f"GPU in use by running job (PID {gpu_worker_pid}). Wait for completion or cancel job.",
                current_worker=gpu_worker,
                conflict=gpu_conflict
            )
        else:
            # Worker is idle but has resource loaded - can reload to switch
            if verbose:
                print(f"Worker is idle, switching to {plugin_name}")
                print(f"Action: RELOAD_PLUGIN ({plugin_name})")
                print("=== Resource Validation Complete ===\n")
            return ValidationResult(
                action=ValidationAction.RELOAD_PLUGIN,
                can_proceed=True,
                message=f"Switching from idle plugin on worker {gpu_worker_pid} to {plugin_name}.",
                current_worker=gpu_worker,
                plugin_name_to_reload=plugin_name,
                new_config=plugin_config,
                conflict=gpu_conflict
            )

    # Case 3: GPU busy with external process
    elif gpu_conflict.status == ResourceStatus.EXTERNAL_BUSY:
        if verbose:
            print("GPU Status: EXTERNAL_BUSY")
        external_processes = gpu_conflict.external_processes
        process_names = [p.get('name', 'unknown') for p in external_processes[:3]]
        if verbose:
            print(f"✗ External processes using GPU: {', '.join(process_names)}")
            for proc in external_processes[:3]:
                print(f"  - PID {proc.get('pid')}: {proc.get('name')} ({proc.get('gpu_memory_mb', 0)} MB)")
            print("Action: USER_INTERVENTION")
            print("=== Resource Validation Complete ===\n")

        return ValidationResult(
            action=ValidationAction.USER_INTERVENTION,
            can_proceed=False,
            message=f"GPU in use by external process(es): {', '.join(process_names)}. User intervention required.",
            conflict=gpu_conflict
        )

    # Fallback
    if verbose:
        print("⚠ Unknown GPU status")
        print("Action: ABORT")
        print("=== Resource Validation Complete ===\n")
    return ValidationResult(
        action=ValidationAction.ABORT,
        can_proceed=False,
        message="Unknown GPU status. Cannot proceed.",
        conflict=gpu_conflict
    )

The validation function uses dependency injection to avoid tight coupling. You provide helper functions that know how to work with your specific plugin registry implementation.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()